**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Open Science Platform Notebook 1 -User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

### Sequence Diagram

```mermaid

%%{
  init: {
    'theme': 'base',
    'themeVariables': {
      'primaryColor': '#ffffff',
      'primaryTextColor': '#000000',
      'primaryBorderColor': '#000000',
      'lineColor': '#000000',
      'secondaryColor': '#f4f4f4',
      'secondaryTextColor': '#000000',
      'tertiaryColor': '#d3d3d3',
      'tertiaryTextColor': '#000000',
      'background': '#ffffff',
      'actorBkg': '#B4B4B4',
      'actorTextColor': '#000000',
      'actorBorder': '#000000',
      'actorLineColor': '#000000',
      'signalColor': '#000000',
      'signalTextColor': '#000000',
      'activationBorderColor': '#000000',
      'activationBkgColor': '#d3d3d3',
      'sequenceNumberColor': '#000000',
      'noteBkgColor': '#F0F0F0',
      'noteTextColor': '#000000',
      'noteBorderColor': '#000000'
    }
  }
}%%

sequenceDiagram
    participant Platform as "Platform"
    participant Blockchain as "Iroha 1 Blockchain"

    Note over Platform, Blockchain: Deploy smart contract for account creation
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully
    
    Note over Platform, Blockchain: Account creation
    Platform->>Blockchain: Create Account
    Blockchain-->>Platform: Account Created Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm account creation
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
    
    
    Note over Platform, Blockchain: Deploy smart contract for details setting
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully

    Note over Platform, Blockchain: Set user details
    Platform->>Blockchain: Set User Details in Blockchain
    Blockchain-->>Platform: User Details Set Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm details setting
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
```

In [46]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_user_json_ld_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [47]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [48]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-01-24 02:54:34,627 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-01-24 02:54:34.627 | INFO     | __main__:<module>:3 - b'764e13fd1afdd09c0d3ec9ca35f1d529c0df4c4c2b37676f8d84f0171ea069bc'


2 - Creates a user account into the blockchain.

In [56]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-01-24 10:50:16.217 | INFO     | __main__:<module>:13 - User Role: author
2025-01-24 10:50:16.220 | INFO     | __main__:<module>:24 - Creating account with name: boring_keller
2025-01-24 10:50:16.222 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"


2025-01-24 10:50:16.242 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'764e13fd1afdd09c0d3ec9ca35f1d529c0df4c4c2b37676f8d84f0171ea069bc'
2025-01-24 10:50:16.243 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 91ABF7E328C683B7454BCA7044F51D6925A88708
2025-01-24 10:50:16.244 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-24 10:50:16.246 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"
2025-01-24 10:50:16,262 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-24 10:50:16,264 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-24 10:50:18,385 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-24 10:50:18,415 - INFO - ('COMMITTED', 5, 0)
2025-01-24 10:50:18.424 | INFO     | dump_to_json:dump_to_json_ld:82 - Appended new entry to file 'datasets/accounts.json', current total entries: 78
2025-01-24 10:50:1

b'76351794cb5b196e253813925b0821a5077d06822b103f9cc1131fc3ac658951'

3 - Queries the blockchain to confirm the succesful creation of the account.

In [59]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-01-24 10:57:35.110 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-24 10:57:35,113 - WARNING - create_account_contract_address: 91ABF7E328C683B7454BCA7044F51D6925A88708
2025-01-24 10:57:35,138 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-24 10:57:35,141 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-24 10:57:36,986 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-24 10:57:37,017 - INFO - ('COMMITTED', 5, 0)
2025-01-24 10:57:37,019 - WARNING - Account Data: {'address': '91ABF7E328C683B7454BCA7044F51D6925A88708', 'hex_hash': '35a1a0cc3dc25a3d163186aa4f110843d6648da6a12a632f7f7d63b2d4dcff70'}
2025-01-24 10:57:37.020 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-24 10:57:37,024 - INFO - Appended data under 'boring_keller@test' in logs/account_data.json
2025-01-24 10:57:37.025 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-24 10:57:37.026 | DEBUG    | integr

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [51]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-01-24 02:54:37,905 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-01-24 02:54:37.907 | INFO     | __main__:<module>:3 - b'8765694a4ad50ce0bcbd2aee1f70a338d3a9d44018bb518617875915a4a59840'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [55]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-01-24 10:46:20.622 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-24 10:46:20.650 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'8765694a4ad50ce0bcbd2aee1f70a338d3a9d44018bb518617875915a4a59840'
2025-01-24 10:46:20.652 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 7ABFBB7ACEC6BEC5B631C333A54CEC1B52BD8687
2025-01-24 10:46:20.653 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-24 10:46:20.655 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-01-24 10:46:20,659 - INFO - Processing account: compassionate_raman@test
2025-01-24 10:46:20,661 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Compassionate Raman', 'foaf:mbox': 'compassionate_raman@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'University of Bolton'},

6 - Queries the blockchain again to confirm the succesfull details setting.

In [53]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
user_json_ld_cid = process_raw_data(details)

2025-01-24 02:54:40.952 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-24 02:54:40,956 - WARNING - create_account_contract_address: 91ABF7E328C683B7454BCA7044F51D6925A88708
2025-01-24 02:54:40,972 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-24 02:54:40,974 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-24 02:54:43,933 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-24 02:54:43,961 - INFO - ('COMMITTED', 5, 0)
2025-01-24 02:54:43,963 - WARNING - Account Data: {'address': '91ABF7E328C683B7454BCA7044F51D6925A88708', 'hex_hash': '766db3e7d2a0153cb87fa2ad1278c666cdeb955c9d048cd36e33fb6f7fb9860e'}
2025-01-24 02:54:43.965 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-24 02:54:43,969 - INFO - Appended data under 'compassionate_raman@test' in logs/account_data.json
2025-01-24 02:54:43.970 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-24 02:54:43.971 | DEBUG    | 

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [54]:
user_metadata = download_json_from_ipfs(user_json_ld_cid)
logger.info(f"User Metadata: {user_metadata}")

2025-01-24 02:54:44.019 | INFO     | __main__:<module>:2 - User Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'compassionate_raman@test', 'schema:publicKey': '86b16fdb66d1e59447ab0be9ecc898b6fe8b3c14d840408a75dccad7c77fb063', 'schema:roleName': 'author'}, 'foaf:mbox': 'compassionate_raman@email.com', 'foaf:name': 'Compassionate Raman', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'University of Bolton'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '0426-7964-4964-X'}}
